In [23]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from city_time_interval import latitude, longitude, start_date, end_date


Latitude: 42.3554334, Longitude: -71.060511
The date 5 years ago from 2025-08-24 was: 2020-08-24


In [24]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [25]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": latitude,
	"longitude": longitude,
	"start_date": start_date,
	"end_date": end_date,
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"timezone": "America/New_York",
}
responses = openmeteo.weather_api(url, params=params)

In [26]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

Coordinates: 42.35500717163086°N -71.12905883789062°E
Elevation: 10.0 m asl
Timezone: b'America/New_York'b'GMT-4'
Timezone difference to GMT+0: -14400s


In [27]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()


In [28]:
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

In [29]:
print(type(daily_data))

<class 'dict'>


In [30]:
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum

In [31]:
daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe["date"] = daily_dataframe["date"].dt.date


In [32]:
daily_dataframe.head(n=5)

,date,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2020-08-24,31.990999,20.941000,2.0
1,2020-08-25,32.690998,21.341000,0.2
2,2020-08-26,24.941000,15.790999,0.0
3,2020-08-27,22.890999,14.040999,15.4
4,2020-08-28,27.091000,15.341000,0.0
